In [1]:
import pandas as pd
import numpy as np
import random
import sklearn as sk

from sklearn import linear_model
from sklearn.model_selection import cross_val_predict
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split

In [63]:
#Read in data and preprocessing 
df = pd.read_csv('prediction_resources/resources/data clean/prediction.csv',index_col='code')
df = df.rename(columns={"2016": "2016_g"})
df["2016_total"] = df["2016_g"]+df["2016_b"]+df["2016_s"]

In [64]:
df.fillna(df.mean(), inplace=True)

In [68]:
#delete missing data
filtered_df = df[~np.isnan(df['2012_CPI']) & (~np.isnan(df['2012_revenue']))]
#normalization 
norm_factor = ['2012_CPI','2012_GDP','2012_POP','2012_revenue','2012_rev/pop','2008_CPI','2008_GDP','2008_POP','2008_revenue','2008_rev/pop','2008/TOTAL','2012/TOTAL']
normed_df = (filtered_df[norm_factor] - filtered_df[norm_factor].mean())/filtered_df[norm_factor].std()
normed_df[['2008','2012','2016_g','2008_s','2012_s','2016_s','2008_b','2012_b','2016_b','2008_r','2012_r','2016_r','HOST','2016_total',]]=filtered_df[['2008','2012','2016_g','2008_s','2012_s','2016_s','2008_b','2012_b','2016_b','2008_r','2012_r','2016_r','HOST','2016_total']]

In [69]:
df.head()

,2008,2012,2016_g,2012_CPI,2012_GDP,2012_POP,2012_revenue,2012_rev/pop,2008_CPI,2008_GDP,...,2008_b,2012_b,2016_b,2008_r,2012_r,2016_r,2008_s,2012_s,2016_s,2016_total
code,,,,,,,,,,,,,,,,,,,,,
AFG,0,0,0,7.218,2.053654e+10,29726803,7.989000,2.690000e-07,30.555,1.019053e+10,...,1,1,0,75,79,88,0,0,0,0
ARG,20,1,3,10.030,5.490000e+11,42095224,27.969238,2.996018e-06,8.584,3.630000e+11,...,31,2,0,11,43,27,0,1,1,4
ARM,0,0,1,2.555,1.061932e+10,2978339,22.860000,7.680000e-06,8.950,1.166204e+10,...,6,2,0,73,61,42,0,1,3,4
AUS,31,8,8,1.763,1.540000e+12,22728254,26.596000,1.170000e-06,4.353,1.050000e+12,...,76,12,10,7,8,10,42,17,11,29
AUT,0,0,0,2.486,4.070000e+11,8429991,43.565000,5.170000e-06,3.216,4.280000e+11,...,2,0,1,65,86,78,1,0,0,1


In [107]:
#split data into training and test sets
y = df[['2016_total','2016_g','2016_s','2016_b']]
X = df.drop(['2016_total','2016_g','2016_s','2016_b'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

yn = normed_df[['2016_total','2016_g','2016_s','2016_b']]
Xn = normed_df.drop(['2016_total','2016_g','2016_s','2016_b'], axis=1)
Xn_train, Xn_test, yn_train, yn_test = train_test_split(Xn, yn, test_size=0.15, random_state=42)

#sklearn creat ridge linear model 
reg = linear_model.Lasso(alpha = .3,fit_intercept = True)

In [108]:
X_test

,2008,2012,2012_CPI,2012_GDP,2012_POP,2012_revenue,2012_rev/pop,2008_CPI,2008_GDP,2008_POP,...,HOST,2008Z,2012Z,2008_b,2012_b,2008_r,2012_r,2016_r,2008_s,2012_s
code,,,,,,,,,,,,,,,,,,,,,
JAM,9,4,6.898000,1.474642e+10,2707805,31.702000,1.170000e-05,22.021000,1.367855e+10,2671934,...,0,-0.043145,-0.070227,5,4,17,18,16,3,4
SWE,0,1,0.888000,5.440000e+11,9519374,33.383000,3.510000e-06,3.437000,5.140000e+11,9219637,...,0,-0.466752,-0.396441,2,3,57,37,29,5,4
ITA,8,8,3.041000,2.070000e+12,59539717,36.590000,6.150000e-07,3.375000,2.390000e+12,58826731,...,0,-0.090213,0.364726,20,11,18,9,9,14,9
TUN,1,2,5.138000,4.504418e+10,10777500,29.179000,2.710000e-06,4.921000,4.485659e+10,10328900,...,0,-0.419685,-0.287703,0,1,52,35,75,0,0
IRL,0,1,1.693000,2.250000e+11,4586897,31.872000,6.950000e-06,4.054000,2.750000e+11,4489544,...,0,-0.466752,-0.396441,2,4,65,42,62,1,1
POL,6,2,3.557000,5.000000e+11,38063164,32.662000,8.580000e-07,4.349000,5.300000e+11,38125759,...,0,-0.184347,-0.287703,1,6,21,30,33,13,2
CUB,2,5,5.233413,7.313905e+10,11342631,27.969238,2.996018e-06,9.674413,6.080630e+10,11290239,...,0,-0.372617,0.038511,11,7,31,16,18,34,3
FRA,25,11,1.956000,2.680000e+12,65659790,42.378000,6.450000e-07,2.814000,2.920000e+12,64374990,...,0,0.709934,0.690940,28,12,9,7,7,23,11


In [109]:
y_test

,2016_total,2016_g,2016_s,2016_b
code,,,,
JAM,11,6,3,2
SWE,11,2,6,3
ITA,28,8,12,8
TUN,3,0,0,3
IRL,2,0,2,0
POL,11,2,3,6
CUB,11,5,2,4
FRA,42,10,18,14


In [110]:
regr = linear_model.LinearRegression()
regr.fit(Xn_train, yn_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [111]:
print(regr.coef_)

[[-1.31926808e-01  1.90446908e+01  1.51969297e+02 -4.70576313e-03
  -1.03523091e+01 -8.92911018e-02 -9.03404262e+00 -1.56837770e+02
  -3.17057169e-02  1.09716853e+01 -8.17940884e+07  4.78556778e+07
   3.84983809e+06 -5.20373101e+06  7.07590451e-02  1.33468331e-01
   1.82640247e-02  7.73673259e-01 -2.48961674e-02  5.94776679e-02
  -1.18909453e-01 -4.70697063e+00]
 [-1.95601685e-01 -5.76469048e+00  4.19853509e+01 -2.45143872e-01
   5.69831397e-01 -1.14321460e-01  6.07378898e+00 -4.27424525e+01
  -1.60385354e-01 -4.52248193e-01 -4.22812231e+07  1.87119935e+07
   1.99006911e+06 -2.03470492e+06 -6.71620982e-02  5.72918910e-01
  -7.40027150e-02 -1.36398349e-01  1.13771691e-02  6.90768114e-02
  -9.01207370e-02  5.54575354e+00]
 [ 5.89714032e-02 -3.80021204e+00  2.90556433e+01  1.68725695e-01
  -1.20593793e+01  2.75334013e-01  6.63448965e+00 -2.95052804e+01
  -3.40196381e-01  1.27425227e+01 -1.71736476e+07  4.07669817e+07
   8.08319465e+05 -4.43292101e+06 -4.27978324e-02  4.37112938e-02
   1.1

In [112]:
print("--Max")
for k in regr.coef_:
    print( sorted( [(x,i) for (i,x) in enumerate(k)], reverse=True )[:5] )
print("--Min")
for k in regr.coef_:
    print( sorted( [(x,i) for (i,x) in enumerate(k)], reverse=False )[:5] )

--Max
[(47855677.78582696, 11), (3849838.0868229433, 12), (151.96929673516334, 2), (19.044690763725022, 1), (10.971685261024563, 9)]
[(18711993.516034342, 11), (1990069.1084929283, 12), (41.98535088576228, 2), (6.073788982436685, 6), (5.545753540857446, 21)]
[(40766981.70333128, 11), (808319.4648571471, 12), (29.055643326013087, 2), (12.742522726130115, 9), (6.634489647250169, 6)]
[(1263894.9167916367, 13), (1051449.5134728663, 12), (80.92830251894671, 2), (28.609593279382814, 1), (1.1372388576029322, 4)]
--Min
[(-81794088.3739294, 10), (-5203731.006309513, 13), (-156.8377696855555, 7), (-10.352309067178043, 4), (-9.03404262328785, 6)]
[(-42281223.10989769, 10), (-2034704.9175190534, 13), (-42.74245254591046, 7), (-5.764690480301804, 1), (-0.4522481927046158, 9)]
[(-17173647.60021147, 10), (-4432921.005582108, 13), (-29.505280437508908, 7), (-12.059379324129178, 4), (-3.8002120436576456, 1)]
[(-22339217.663820177, 10), (-11623297.433538547, 11), (-84.59003669849218, 7), (-21.7423212520

In [113]:
Significatif = [12,13,2,7,11,1]
Significatif

[12, 13, 2, 7, 11, 1]

In [114]:
[ Xn_train.columns[k] for k in Significatif]

['2008', '2012', '2012_POP', '2008_POP', '2012/TOTAL', '2012_GDP']

## Prédiction

### Estimations faites sur l'ensemble de tests

In [140]:
pred_yn_test = regr.predict(Xn)
prediction = pd.DataFrame(pred_yn_test).set_index(Xn.index)
prediction.columns = ['TotalMed', 'EstimGold','EstimSilver','EstimBronz']
prediction.reset_index().merge(yn.reset_index(),on="code")

,code,TotalMed,EstimGold,EstimSilver,EstimBronz,2016_total,2016_g,2016_s,2016_b
0,AFG,-0.477093,-0.565217,0.084598,0.003527,0,0,0,0
1,ARG,3.996510,2.368815,1.807756,-0.180061,4,3,1,0
2,ARM,5.194207,2.268892,2.790343,0.134973,4,1,3,0
3,AUS,29.574350,8.302146,10.869624,10.402581,29,8,11,10
4,AUT,1.799336,0.031985,0.436836,1.330515,1,0,0,1
5,AZE,12.986692,0.774054,4.999136,7.213501,18,1,7,10
6,BEL,8.778048,2.759250,2.131880,3.886918,6,2,2,2
7,BLR,8.630105,1.131247,3.507845,3.991014,9,1,4,4
8,BRA,19.000000,7.000000,6.000000,6.000000,19,7,6,6
9,CAN,20.933513,4.213441,2.993352,13.726721,22,4,3,15


### Information les plus importantes dans ce modèle

In [138]:
", ".join([ Xn_train.columns[k] for k in Significatif])

'2008, 2012, 2012_POP, 2008_POP, 2012/TOTAL, 2012_GDP'